# Confusion Matrix - CNN (AE Min 8 Node)

In [1]:
import numpy as np
import pickle
from keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


In [2]:
base_dir = "..\\..\\..\\data\\BLE_data\\"
def read_test_data(set_num, model_name, mode):
    if model_name == 5:
        raw_data = np.loadtxt(base_dir + "raw_data\\in_" + mode + "_" + str(set_num) + '.csv', delimiter=',', dtype=np.float32)
    elif model_name == 6:
        raw_data = np.loadtxt(base_dir + "raw_data\\out_" + mode + "_" + str(set_num) + '.csv', delimiter=',', dtype=np.float32)
    b_data    = raw_data[:,0:3]
    cell_data = raw_data[:,3:13]

    return b_data, cell_data

def read_denoised_test_data(set_num, model_name, mode):
    if model_name == 5:
        raw_data = np.loadtxt(base_dir + "filtered_data\\in_" + mode + "_" + str(set_num) + '_denoised.csv', delimiter=',', dtype=np.float32)
    elif model_name == 6:
        raw_data = np.loadtxt(base_dir + "filtered_data\\out_" + mode + "_" + str(set_num) + '_denoised.csv', delimiter=',', dtype=np.float32)
    b_data    = raw_data[:,0:3]
    cell_data = raw_data[:,3:13]

    return b_data, cell_data

def one_hot_convert_normal(data, end_line, output_num):
    list_data = []

    for row in range(0, end_line):
        for column in range(0, output_num):
            if data[row][column] == 1:
                list_data.append(column + 1)
    return list_data

In [3]:
# model_name: 5 (car_in) / 6 (car_out)
# Car-in data
model_name = 5
set_num = 6000
train_set_num = int(set_num * 0.75)
test_set_num = int(set_num * 0.25)

x_in_train_noisy, y_in_train_noisy = read_test_data(train_set_num, model_name, "train")
x_in_train_denoised, y_in_train_denoised = read_denoised_test_data(train_set_num, model_name, "train")
x_in_test_noisy, y_in_test_noisy = read_test_data(test_set_num, model_name, "test")
x_in_test_denoised, y_in_test_denoised = read_denoised_test_data(test_set_num, model_name, "test")

In [4]:
# model_name: 5 (car_in) / 6 (car_out)
# Car-out data
model_name = 6
set_num = 6000
train_set_num = int(set_num * 0.75)
test_set_num = int(set_num * 0.25)

x_out_train_noisy, y_out_train_noisy = read_test_data(train_set_num, model_name, "train")
x_out_train_denoised, y_out_train_denoised = read_denoised_test_data(train_set_num, model_name, "train")
x_out_test_noisy, y_out_test_noisy = read_test_data(test_set_num, model_name, "test")
x_out_test_denoised, y_out_test_denoised = read_denoised_test_data(test_set_num, model_name, "test")

In [5]:
# Car-in
ae_x_in_train_noisy = x_in_train_noisy.astype('float32') / -100.
ae_x_in_train_denoised = x_in_train_denoised.astype('float32') / -100.
ae_x_in_test_noisy = x_in_test_noisy.astype('float32') / -100.
ae_x_in_test_denoised = x_in_test_denoised.astype('float32') / -100.
# Car-out
ae_x_out_train_noisy = x_out_train_noisy.astype('float32') / -100.
ae_x_out_train_denoised = x_out_train_denoised.astype('float32') / -100.
ae_x_out_test_noisy = x_out_test_noisy.astype('float32') / -100.
ae_x_out_test_denoised = x_out_test_denoised.astype('float32') / -100.

In [6]:
# Car-in
x_in_train_noisy = x_in_train_noisy.reshape(45000, 3, 1)
x_in_train_denoised = x_in_train_denoised.reshape(45000, 3, 1)
x_in_test_noisy = x_in_test_noisy.reshape(15000, 3, 1)
x_in_test_denoised = x_in_test_denoised.reshape(15000, 3, 1)
# Car-out
x_out_train_noisy = x_out_train_noisy.reshape(45000, 3, 1)
x_out_train_denoised = x_out_train_denoised.reshape(45000, 3, 1)
x_out_test_noisy = x_out_test_noisy.reshape(15000, 3, 1)
x_out_test_denoised = x_out_test_denoised.reshape(15000, 3, 1)

In [7]:
# Training data
output_num = 10
end_line = train_set_num * output_num

oh_y_in_train_denoised = one_hot_convert_normal(y_in_train_denoised, end_line, output_num)
oh_y_in_train_denoised = np.array(oh_y_in_train_denoised)

oh_y_out_train_denoised = one_hot_convert_normal(y_out_train_denoised, end_line, output_num)
oh_y_out_train_denoised = np.array(oh_y_out_train_denoised)

# Testing data
end_line = test_set_num * output_num

oh_y_in_test_denoised = one_hot_convert_normal(y_in_test_denoised, end_line, output_num)
oh_y_in_test_denoised = np.array(oh_y_in_test_denoised)

oh_y_out_test_denoised = one_hot_convert_normal(y_out_test_denoised, end_line, output_num)
oh_y_out_test_denoised = np.array(oh_y_out_test_denoised)

In [8]:
autoencoder_01 = load_model('autoencoder_01.h5')
autoencoder_02 = load_model('autoencoder_02.h5')
autoencoder_03 = load_model('autoencoder_03.h5')
autoencoder_04 = load_model('autoencoder_04.h5')

# [ 방법 1 ]
## 데이터 10개의 평균 사용

In [9]:
import numpy as np

def average(x, y, set_num, data_num):
    idx = 0
    sum = np.array([0, 0, 0])
    data = [[0 for col in range(3)] for row in range(int(set_num * 10 / data_num))]
    label = [[0 for col in range(10)] for row in range(int(set_num * 10 / data_num))]
    for i in range(set_num * 10):
        for j in range(3):
            sum[j] += x[i][j]
        if(i % data_num == data_num - 1):
            for j in range(3):
                data[idx][j] = int(round(sum[j] / data_num))
            for j in range(10):
                label[idx][j] = y[i][j]
            idx += 1
            sum = np.array([0, 0, 0])    
    return data, label

In [10]:
after_autoencoder_x_train = autoencoder_01.predict(ae_x_in_train_noisy) * (-100)
x_train, y_train = average(after_autoencoder_x_train, y_in_train_noisy, 4500, 10)

In [11]:
after_autoencoder_x_test = autoencoder_01.predict(ae_x_in_test_noisy) * (-100)
x_test, y_test = average(after_autoencoder_x_test, y_in_test_noisy, 1500, 10)

In [12]:
x_train = np.array(x_train)
x_test = np.array(x_test)
ae_x_in_train = x_train.astype('float32') / -100.
ae_x_in_test = x_test.astype('float32') / -100.

In [13]:
# Average data
# Train data
oh_y_in_train = one_hot_convert_normal(y_train, int(4500 * output_num / 10), output_num)
oh_y_in_train = np.array(oh_y_in_train)

# Test data
oh_y_in_test = one_hot_convert_normal(y_test, int(1500 * output_num / 10), output_num)
oh_y_in_test = np.array(oh_y_in_test)

## CNN 01

In [14]:
cnn_01 = load_model('cnn_01.h5')

print("cnn_01\n")

print("test 01\n")
after_autoencoder_x_train = autoencoder_04.predict(ae_x_in_train_denoised) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(ae_x_in_test_denoised) * (-100)
after_autoencoder_x_train = after_autoencoder_x_train.reshape(45000, 3, 1)
after_autoencoder_x_test = after_autoencoder_x_test.reshape(15000, 3, 1)

prediction = cnn_01.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction.argmax(axis=1)+1)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = cnn_01.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction.argmax(axis=1)+1)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 02\n")
prediction = cnn_01.predict(x_out_train_denoised)
cm = confusion_matrix(oh_y_out_train_denoised, prediction.argmax(axis=1)+1)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = cnn_01.predict(x_out_test_denoised)
cm = confusion_matrix(oh_y_out_test_denoised, prediction.argmax(axis=1)+1)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 03\n")
after_autoencoder_x_train = autoencoder_02.predict(ae_x_out_train_noisy) * (-100)
after_autoencoder_x_test = autoencoder_02.predict(ae_x_out_test_noisy) * (-100)
after_autoencoder_x_train = after_autoencoder_x_train.reshape(45000, 3, 1)
after_autoencoder_x_test = after_autoencoder_x_test.reshape(15000, 3, 1)

prediction = cnn_01.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction.argmax(axis=1)+1)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = cnn_01.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction.argmax(axis=1)+1)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 04\n")
after_autoencoder_x_train = autoencoder_03.predict(ae_x_in_train_noisy) * (-100)
after_autoencoder_x_test = autoencoder_03.predict(ae_x_in_test_noisy) * (-100)
after_autoencoder_x_train = after_autoencoder_x_train.reshape(45000, 3, 1)
after_autoencoder_x_test = after_autoencoder_x_test.reshape(15000, 3, 1)

prediction = cnn_01.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction.argmax(axis=1)+1)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = cnn_01.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction.argmax(axis=1)+1)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 05\n")
after_autoencoder_x_train = autoencoder_01.predict(ae_x_in_train_noisy)
after_autoencoder_x_test = autoencoder_01.predict(ae_x_in_test_noisy)
after_autoencoder_x_train = autoencoder_04.predict(after_autoencoder_x_train) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(after_autoencoder_x_test) * (-100)
after_autoencoder_x_train = after_autoencoder_x_train.reshape(45000, 3, 1)
after_autoencoder_x_test = after_autoencoder_x_test.reshape(15000, 3, 1)

prediction = cnn_01.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction.argmax(axis=1)+1)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = cnn_01.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction.argmax(axis=1)+1)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 06\n")
after_autoencoder_x_train = autoencoder_04.predict(ae_x_in_train) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(ae_x_in_test) * (-100)
after_autoencoder_x_train = after_autoencoder_x_train.reshape(4500, 3, 1)
after_autoencoder_x_test = after_autoencoder_x_test.reshape(1500, 3, 1)

prediction = cnn_01.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_in_train, prediction.argmax(axis=1)+1)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = cnn_01.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_in_test, prediction.argmax(axis=1)+1)
print("Testing data Confusion Matrix")
print(cm)
print("\n")

cnn_01

test 01

Training data Confusion Matrix
[[   0 4500    0    0    0    0    0    0    0    0]
 [   0 4461    0    0    0    0   16   10    0   13]
 [   0    0    0    0    0    0    0    0    0 4500]
 [   0  174    0    0    0    0    0    0    0 4326]
 [   0    0    0    0    0    0    0    0    0 4500]
 [   0    0    0    0    0    0    0    0    0 4500]
 [ 104  581    0  684    0    0 2812  273    0   46]
 [4486    0    0    0    0    0    5    0    0    9]
 [   0    0    0    0    0    0    0    0    0 4500]
 [   0 4500    0    0    0    0    0    0    0    0]]


Testing data Confusion Matrix
[[   0 1500    0    0    0    0    0    0    0    0]
 [   0 1479    0    0    0    0    4    8    0    9]
 [   0    0    0    0    0    0    0    0    0 1500]
 [   0   61    0    0    0    0    0    0    0 1439]
 [   0    0    0    0    0    0    0    0    0 1500]
 [   0    0    0    0    0    0    0    0    0 1500]
 [  34  212    0  224    0    0  921   95    0   14]
 [1492    0    0  

## CNN 02

In [15]:
cnn_02 = load_model('cnn_02.h5')

print("cnn_02\n")

print("test 01\n")
after_autoencoder_x_train = autoencoder_04.predict(ae_x_in_train_denoised) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(ae_x_in_test_denoised) * (-100)
after_autoencoder_x_train = after_autoencoder_x_train.reshape(45000, 3, 1)
after_autoencoder_x_test = after_autoencoder_x_test.reshape(15000, 3, 1)

prediction = cnn_02.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction.argmax(axis=1)+1)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = cnn_02.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction.argmax(axis=1)+1)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 02\n")
prediction = cnn_02.predict(x_out_train_denoised)
cm = confusion_matrix(oh_y_out_train_denoised, prediction.argmax(axis=1)+1)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = cnn_02.predict(x_out_test_denoised)
cm = confusion_matrix(oh_y_out_test_denoised, prediction.argmax(axis=1)+1)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 03\n")
after_autoencoder_x_train = autoencoder_02.predict(ae_x_out_train_noisy) * (-100)
after_autoencoder_x_test = autoencoder_02.predict(ae_x_out_test_noisy) * (-100)
after_autoencoder_x_train = after_autoencoder_x_train.reshape(45000, 3, 1)
after_autoencoder_x_test = after_autoencoder_x_test.reshape(15000, 3, 1)

prediction = cnn_02.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction.argmax(axis=1)+1)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = cnn_02.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction.argmax(axis=1)+1)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 04\n")
after_autoencoder_x_train = autoencoder_03.predict(ae_x_in_train_noisy) * (-100)
after_autoencoder_x_test = autoencoder_03.predict(ae_x_in_test_noisy) * (-100)
after_autoencoder_x_train = after_autoencoder_x_train.reshape(45000, 3, 1)
after_autoencoder_x_test = after_autoencoder_x_test.reshape(15000, 3, 1)

prediction = cnn_02.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction.argmax(axis=1)+1)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = cnn_02.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction.argmax(axis=1)+1)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 05\n")
after_autoencoder_x_train = autoencoder_01.predict(ae_x_in_train_noisy)
after_autoencoder_x_test = autoencoder_01.predict(ae_x_in_test_noisy)
after_autoencoder_x_train = autoencoder_04.predict(after_autoencoder_x_train) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(after_autoencoder_x_test) * (-100)
after_autoencoder_x_train = after_autoencoder_x_train.reshape(45000, 3, 1)
after_autoencoder_x_test = after_autoencoder_x_test.reshape(15000, 3, 1)

prediction = cnn_02.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction.argmax(axis=1)+1)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = cnn_02.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction.argmax(axis=1)+1)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 06\n")
after_autoencoder_x_train = autoencoder_04.predict(ae_x_in_train) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(ae_x_in_test) * (-100)
after_autoencoder_x_train = after_autoencoder_x_train.reshape(4500, 3, 1)
after_autoencoder_x_test = after_autoencoder_x_test.reshape(1500, 3, 1)

prediction = cnn_02.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_in_train, prediction.argmax(axis=1)+1)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = cnn_02.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_in_test, prediction.argmax(axis=1)+1)
print("Testing data Confusion Matrix")
print(cm)
print("\n")

cnn_02

test 01

Training data Confusion Matrix
[[4348   71   14    0    0    0   39    0    0   28]
 [ 138 3892   10    0    0    0  150    0    0  310]
 [   0    0 4492    0    0    0    0    0    8    0]
 [ 157   11    6 4173  137    0    0    0   16    0]
 [   0    0    0   50 4168   20    0    0  262    0]
 [   0    0    0    0    0 4312    0    0  188    0]
 [   0   45    0    0    0    0 4436    0   13    6]
 [   0    0    0    0    0    0   32 4468    0    0]
 [   0    0    4   52  274  191    4    0 3975    0]
 [   0  117    0    0    0    0    0    0    0 4383]]


Testing data Confusion Matrix
[[1449   22   11    0    0    0   10    0    0    8]
 [  52 1269    5    0    0    0   52    0    0  122]
 [   0    0 1498    0    0    0    0    0    2    0]
 [  54    4    4 1383   49    0    0    0    6    0]
 [   0    0    0   12 1387    4    0    0   97    0]
 [   0    0    0    0    0 1447    0    0   53    0]
 [   0   16    0    0    0    0 1477    0    5    2]
 [   0    0    0  

## CNN 03

In [16]:
cnn_03 = load_model('cnn_03.h5')

print("cnn_03\n")

print("test 01\n")
after_autoencoder_x_train = autoencoder_04.predict(ae_x_in_train_denoised) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(ae_x_in_test_denoised) * (-100)
after_autoencoder_x_train = after_autoencoder_x_train.reshape(45000, 3, 1)
after_autoencoder_x_test = after_autoencoder_x_test.reshape(15000, 3, 1)

prediction = cnn_03.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction.argmax(axis=1)+1)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = cnn_03.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction.argmax(axis=1)+1)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 02\n")
prediction = cnn_03.predict(x_out_train_denoised)
cm = confusion_matrix(oh_y_out_train_denoised, prediction.argmax(axis=1)+1)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = cnn_03.predict(x_out_test_denoised)
cm = confusion_matrix(oh_y_out_test_denoised, prediction.argmax(axis=1)+1)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 03\n")
after_autoencoder_x_train = autoencoder_02.predict(ae_x_out_train_noisy) * (-100)
after_autoencoder_x_test = autoencoder_02.predict(ae_x_out_test_noisy) * (-100)
after_autoencoder_x_train = after_autoencoder_x_train.reshape(45000, 3, 1)
after_autoencoder_x_test = after_autoencoder_x_test.reshape(15000, 3, 1)

prediction = cnn_03.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction.argmax(axis=1)+1)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = cnn_03.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction.argmax(axis=1)+1)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 04\n")
after_autoencoder_x_train = autoencoder_03.predict(ae_x_in_train_noisy) * (-100)
after_autoencoder_x_test = autoencoder_03.predict(ae_x_in_test_noisy) * (-100)
after_autoencoder_x_train = after_autoencoder_x_train.reshape(45000, 3, 1)
after_autoencoder_x_test = after_autoencoder_x_test.reshape(15000, 3, 1)

prediction = cnn_03.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction.argmax(axis=1)+1)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = cnn_03.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction.argmax(axis=1)+1)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 05\n")
after_autoencoder_x_train = autoencoder_01.predict(ae_x_in_train_noisy)
after_autoencoder_x_test = autoencoder_01.predict(ae_x_in_test_noisy)
after_autoencoder_x_train = autoencoder_04.predict(after_autoencoder_x_train) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(after_autoencoder_x_test) * (-100)
after_autoencoder_x_train = after_autoencoder_x_train.reshape(45000, 3, 1)
after_autoencoder_x_test = after_autoencoder_x_test.reshape(15000, 3, 1)

prediction = cnn_03.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction.argmax(axis=1)+1)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = cnn_03.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction.argmax(axis=1)+1)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 06\n")
after_autoencoder_x_train = autoencoder_04.predict(ae_x_in_train) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(ae_x_in_test) * (-100)
after_autoencoder_x_train = after_autoencoder_x_train.reshape(4500, 3, 1)
after_autoencoder_x_test = after_autoencoder_x_test.reshape(1500, 3, 1)

prediction = cnn_03.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_in_train, prediction.argmax(axis=1)+1)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = cnn_03.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_in_test, prediction.argmax(axis=1)+1)
print("Testing data Confusion Matrix")
print(cm)
print("\n")

cnn_03

test 01

Training data Confusion Matrix
[[4007  423   14    0    0    0   28    0    0   28]
 [ 109 3939   10    0    0    0  150    0    0  292]
 [   0    0 4492    0    0    0    0    0    8    0]
 [  47  121    6 4130  180    0    0    0   16    0]
 [   0    0    0   45 4264   32    0    0  159    0]
 [   0    0    0    0   12 4338    0    0  150    0]
 [   0   47    0    0    0    0 4432    0   13    8]
 [   0    0    0    0    0    0   32 4468    0    0]
 [   0    0    4   44  397  249    4    0 3802    0]
 [   0  177    0    0    0    0    0    0    0 4323]]


Testing data Confusion Matrix
[[1337  137   11    0    0    0    8    0    0    7]
 [  38 1292    5    0    0    0   52    0    0  113]
 [   0    0 1498    0    0    0    0    0    2    0]
 [  15   43    4 1371   61    0    0    0    6    0]
 [   0    0    0   12 1426    8    0    0   54    0]
 [   0    0    0    0    5 1455    0    0   40    0]
 [   0   18    0    0    0    0 1475    0    5    2]
 [   0    0    0  

## CNN 04

In [17]:
cnn_04 = load_model('cnn_04.h5')

print("cnn_04\n")

print("test 01\n")
after_autoencoder_x_train = autoencoder_04.predict(ae_x_in_train_denoised) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(ae_x_in_test_denoised) * (-100)
after_autoencoder_x_train = after_autoencoder_x_train.reshape(45000, 3, 1)
after_autoencoder_x_test = after_autoencoder_x_test.reshape(15000, 3, 1)

prediction = cnn_04.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction.argmax(axis=1)+1)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = cnn_04.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction.argmax(axis=1)+1)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 02\n")
prediction = cnn_04.predict(x_out_train_denoised)
cm = confusion_matrix(oh_y_out_train_denoised, prediction.argmax(axis=1)+1)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = cnn_04.predict(x_out_test_denoised)
cm = confusion_matrix(oh_y_out_test_denoised, prediction.argmax(axis=1)+1)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 03\n")
after_autoencoder_x_train = autoencoder_02.predict(ae_x_out_train_noisy) * (-100)
after_autoencoder_x_test = autoencoder_02.predict(ae_x_out_test_noisy) * (-100)
after_autoencoder_x_train = after_autoencoder_x_train.reshape(45000, 3, 1)
after_autoencoder_x_test = after_autoencoder_x_test.reshape(15000, 3, 1)

prediction = cnn_04.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction.argmax(axis=1)+1)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = cnn_04.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction.argmax(axis=1)+1)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 04\n")
after_autoencoder_x_train = autoencoder_03.predict(ae_x_in_train_noisy) * (-100)
after_autoencoder_x_test = autoencoder_03.predict(ae_x_in_test_noisy) * (-100)
after_autoencoder_x_train = after_autoencoder_x_train.reshape(45000, 3, 1)
after_autoencoder_x_test = after_autoencoder_x_test.reshape(15000, 3, 1)

prediction = cnn_04.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction.argmax(axis=1)+1)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = cnn_04.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction.argmax(axis=1)+1)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 05\n")
after_autoencoder_x_train = autoencoder_01.predict(ae_x_in_train_noisy)
after_autoencoder_x_test = autoencoder_01.predict(ae_x_in_test_noisy)
after_autoencoder_x_train = autoencoder_04.predict(after_autoencoder_x_train) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(after_autoencoder_x_test) * (-100)
after_autoencoder_x_train = after_autoencoder_x_train.reshape(45000, 3, 1)
after_autoencoder_x_test = after_autoencoder_x_test.reshape(15000, 3, 1)

prediction = cnn_04.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction.argmax(axis=1)+1)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = cnn_04.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction.argmax(axis=1)+1)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 06\n")
after_autoencoder_x_train = autoencoder_04.predict(ae_x_in_train) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(ae_x_in_test) * (-100)
after_autoencoder_x_train = after_autoencoder_x_train.reshape(4500, 3, 1)
after_autoencoder_x_test = after_autoencoder_x_test.reshape(1500, 3, 1)

prediction = cnn_04.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_in_train, prediction.argmax(axis=1)+1)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = cnn_04.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_in_test, prediction.argmax(axis=1)+1)
print("Testing data Confusion Matrix")
print(cm)
print("\n")

cnn_04

test 01

Training data Confusion Matrix
[[3937  514    0    0    0    0   39    0    0   10]
 [  94 4035   10    0    0    0  155    0    0  206]
 [   0    0 4492    0    0    0    0    0    8    0]
 [  35  138    1 4228   79    0    0    0   19    0]
 [   0    0    0  570 3624   20    0    0  286    0]
 [   0    0    0    0    6 4356    0    0  138    0]
 [   0   50    0    0    0    0 4432    0   17    1]
 [   0    0    0    0    0    0   79 4421    0    0]
 [   0    0    4  206  108  262    4    0 3916    0]
 [   0  282    0    0    0    0    0    0    0 4218]]


Testing data Confusion Matrix
[[1314  173    0    0    0    0   10    0    0    3]
 [  34 1338    5    0    0    0   57    0    0   66]
 [   0    0 1498    0    0    0    0    0    2    0]
 [  11   50    1 1394   37    0    0    0    7    0]
 [   0    0    0  194 1202    4    0    0  100    0]
 [   0    0    0    0    2 1459    0    0   39    0]
 [   0   17    0    0    0    0 1474    0    8    1]
 [   0    0    0  